## Dose Response Curve Fitting in Python
### Setting up the stage
While I would naturally fit dose-response curves in R using drc, recently I have started using iPython more and more and was wondering : "Can I do curve fitting in Python directly?"

Turns out this is quite easy, although not necessarily well documented. Let's create a toy example and see how things are done using scipy.optimize & pandas

retreived from: https://gist.github.com/yannabraham/5f210fed773785d8b638?permalink_comment_id=1713810#file-doseresponsepython-ipynb 

In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings

def action_with_warnings():
    warnings.warn("should not appear")

Let's use a 4-parameter sigmoidal function as a starting point:

In [ ]:
def ll4(x,b,c,d,e):
    '''This function is basically a copy of the LL.4 function from the R drc package with
     - b: hill slope
     - c: min response
     - d: max response
     - e: EC50'''
    return(c+(d-c)/(1+np.exp(b*(np.log(x)-np.log(e)))))

def pDose(x):
    '''This is just a helper function, to compute easily log transformed concentrations used in drug discovery'''
    return(-np.log10(1e-6*x))

## Loading my own data

In [ ]:
BaseDir = './results/'

In [ ]:
dfViability = pd.read_csv("{}/viability_scores.csv".format(BaseDir), sep=";", index_col=0)
#dfViability.head()

In [ ]:
dfViability['Metadata_cmpd'].iloc([dfViability['Metadata_pert_type'] == 'trt'])

In [ ]:
# Remove the reference compounds for now
#print(sorted(list(set(dfViability["compound"]))))
compoundList = ['5-Fluo', '5Z-7-O', 'ABT-73', 'AMG 23', 'AZD454', 'AZD776', 'AZD805', 'Abemac', 'Afatin', 'BMS-75', 'Crizot', 'Dabraf', 'Encora', 'Gefiti', 'LGK974', 'LY2603', 'Linsit', 'MK-177', 'MK-220', 'Navito', 'Nutlin', 'Olapar', 'Oxalip', 'PD0325', 'PI-103', 'Paclit', 'Palboc', 'Ruxoli', 'SB2167', 'SB5051', 'SN-38', 'Sapiti', 'Taseli', 'Temozo', 'Tramet', 'Velipa', 'Vinore', 'alpeli', 'binime', 'cobime', 'regora', 'tricir', 'triflu', 'vemura']



In [ ]:
# Remove the reference compounds for now
dfDMSO = dfViability[dfViability.compound.isin(['[dmso]'])]
dfViability = dfViability[dfViability.compound.isin(compoundList)]
dfViability.dropna(inplace=True)
dfViability.reset_index(drop=False, inplace=True)
#dfViability.head()


In [ ]:
# Add the log concentration column
dfViability['logConc'] = pDose(dfViability.conc)
dfViability.head()

## Fitting the curve
is as simple as

In [ ]:
compoundData = drData.groupby(['compound'])
fitData = []
for name,group in compoundData:
    fitCoefs, covMatrix = opt.curve_fit(ll4, group.dose, group.response)
    resids = group.response-group.dose.apply(lambda x: ll4(x,*fitCoefs))
    curFit = dict(zip(['b','c','d','e'],fitCoefs))
    curFit['compound']=name
    curFit['residuals']=sum(resids**2)
    fitData.append(curFit)
fitCompound = [ item['compound'] for item in fitData]

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
fitTable

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
paramTable = pd.DataFrame(params).set_index('compound')[['b','c','d','e']]
paramTable.columns = ['ref_'+i for i in paramTable.columns]
fitTable.join(paramTable)

In [ ]:
refDose = np.linspace(min(drData.logDose)*0.9,max(drData.logDose)*1.1,256)
refDose = (10**-refDose)*1e6
sns.lmplot(x='logDose',y='response',data=drData,hue='compound',fit_reg=False)
for fit in fitData:
    plt.plot([pDose(i) for i in refDose],[ll4(i,*[fit[i] for i in ['b','c','d','e']]) for i in refDose])

## Plotting my own data

In [ ]:
dfDMSO['logConc']  = dfDMSO.conc  * 90
dfDMSO.head()

In [ ]:
compoundData = dfViability.groupby(['compound'])

nrrow = 5 
nrcol = 10
sp = 1

meanDMSO = dfDMSO[['conc','Count_nuclei', 'ObjectNumber_cells', 'logConc']].mean()

fig = plt.figure(figsize = (40,20))
for compound in compoundData:

    meandata = compound[1].groupby('conc').mean()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        mean = meandata.append(meanDMSO,ignore_index=True)

    ax = fig.add_subplot(nrrow, nrcol, sp)
    ax.set_title(compound[0])
    sns.regplot(ax=ax, x='logConc',y='Count_nuclei',data=compound[1],color='gray', fit_reg=False) # plot the compounds 
    sns.regplot(ax=ax, x='logConc',y='Count_nuclei',data=dfDMSO,color='gray', fit_reg=False) # add the DMSO data
    sns.regplot(ax=ax, x='logConc',y='Count_nuclei',data=mean,color='red', fit_reg=False) # add the mean data
    sp +=1
plt.subplots_adjust( wspace=0.4, hspace=0.4)

## Fit one compound at a time

In [ ]:
cmpd = 1 # Select which compound
name = compoundList[cmpd]
print(name)

In [ ]:
#compoundData = drData.groupby(['compound'])
fitData = []
for name,group in compoundData:
    fitCoefs, covMatrix = opt.curve_fit(ll4, group.conc, group.Count_nuclei)
    resids = group.Count_nuclei-group.conc.apply(lambda x: ll4(x,*fitCoefs))
    curFit = dict(zip(['b','c','d','e'],fitCoefs))
    curFit['compound']=name
    curFit['residuals']=sum(resids**2)
    fitData.append(curFit)
fitCompound = [ item['compound'] for item in fitData]

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
fitTable

In [ ]:
params = [{'compound':'5Z-7-O', 'b':1, 'c':0, 'd':100, 'e':0.4,'startDose':10, 'nDose':8, 'dilution':3},]
paramsCompound = [item['compound'] for item in params]

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
paramTable = pd.DataFrame(params).set_index('compound')[['b','c','d','e']]
paramTable.columns = ['ref_'+i for i in paramTable.columns]
fitTable.join(paramTable)